# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Load the dotenv extension
%load_ext dotenv

# Load environment variables from .env
%dotenv

In [2]:
import dask.dataframe as dd

c:\Users\15199\miniconda3\envs\dsi_participant\lib\site-packages\dask\dataframe\_pyarrow_compat.py:15: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

# Load the environment variable PRICE_DATA
ft_dir = os.getenv('PRICE_DATA')

# If recursive is true, the pattern “**” will match any files and zero or more directories, subdirectories and symbolic links to directories.
parquet_files = glob(ft_dir + '**/*.parquet', recursive=True)
parquet_files

['../../05_src/data/prices\\AAPL\\AAPL_2000\\part.0.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2000\\part.1.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2001\\part.0.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2001\\part.1.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2002\\part.0.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2002\\part.1.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2003\\part.0.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2003\\part.1.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2004\\part.0.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2004\\part.1.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2005\\part.0.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2005\\part.1.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2006\\part.0.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2006\\part.1.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2007\\part.0.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2007\\part.1.parquet',
 '../../

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [4]:
# Reading multiple parquet files into a Dask DataFrame
df = dd.read_parquet(parquet_files)

# Grouping by Ticker column and group_keys = False ensures that the group labels do not appear in the index of the result. 
# .apply() will apply the given function to each group. 
# lambda is the other way of defining a function. E.g .shift(1) will take the Close column value from the previous row and assign it to the new column name Close_lag_1.

df_lags = df.groupby('Ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1),
                       Close_lag_2 = x['Adj Close'].shift(1))
)

dd_feat = df_lags.groupby('Ticker', group_keys=False).apply(
    lambda x: x.assign(returns = (x['Close'] / x['Close_lag_1']) - 1,
                       hi_lo_range = x['High'] - x['Low'])
)

C:\Users\15199\AppData\Local\Temp\ipykernel_2180\3633471738.py:8: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  df_lags = df.groupby('Ticker', group_keys=False).apply(
C:\Users\15199\AppData\Local\Temp\ipykernel_2180\3633471738.py:13: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = df_lags.groupby('Ticker', group_keys=False).apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [5]:
# Converting Dask data frame to a pandas data frame. Further calculating moving average using a window of 10 days. First nine values will be NaN as there are not 10 values for the mean.
dd_feat['returns'].compute().rolling(10).mean()

96544          NaN
96608          NaN
96672          NaN
96736          NaN
96800          NaN
            ...   
337898    0.007463
337962    0.006470
338026    0.005422
338090    0.004264
338154    0.003390
Name: returns, Length: 403456, dtype: float64

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

+ It's not necessary to convert to pandas to calculate the moving average return. We can also work with the csv files. Also, repartition() can be used with map_overlap() without converting Dask DataFrame to pandas for calculating the rolling average.  
Dask DataFrame is lazy, meaning it doesn't compute results immediately. It does not show actual data; it just provides an overview. So, the other method is to use .compute() to show actual results to calculate the rolling average, and the compute method converts the dask DataFrame to pandas DataFrame. For data that fits into RAM, pandas can often be faster and easier to use than Dask DataFrame.  

+ It depends on the size of the dataset. If it's small, pandas is the better than Dask. However, if it's large dataset, Dask is the best choice. Our dataset is not very large, so converting to pandas is good choice.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.